
<br> <br> <br> <br>
<br> <br> <br> <br>

<center>


### EAI 6000 Fundamentals of AI
### Assignment 3 Tree Based Models
### Shivangi Vashi
### Anirudh Sharma
### Sandeep Srivastava
### Sweta Mankala


<br> <br> <br> <br>
  

### Northeastern University
### Instructor: Kasun Samarasinghe
### Fall 2020
### 12 November 2020
</center>

******
<br> <br> <br> <br>

<style>
body {
text-align: justify
font-size: 16pt}
</style>


## <center>Introduction  </center>

<br><br>
This week we explore Tree based models. Our data is from: [policing data](https://openpolicing.stanford.edu/data/), and we look at Massachusetts Traffic Stops. Our goal is to explore policing and understand factors that affect the decisions made by the police. In this analysis, we will be performing 
<br><br>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from IPython.display import display

In [2]:
#Importing data
openpolicing_path="../ma_statewide_2020_04_01.csv"

data=pd.read_csv(openpolicing_path,low_memory=False)

In [3]:
data.head()

,raw_row_number,date,location,county_name,subject_age,subject_race,subject_sex,type,arrest_made,citation_issued,...,contraband_weapons,contraband_alcohol,contraband_other,frisk_performed,search_conducted,search_basis,reason_for_stop,vehicle_type,vehicle_registration_state,raw_Race
0,1,2007-06-06,MIDDLEBOROUGH,Plymouth County,33.0,white,male,vehicular,False,True,...,NaN,False,NaN,NaN,False,NaN,Speed,Passenger,MA,White
1,2,2007-06-07,SEEKONK,Bristol County,36.0,white,male,vehicular,False,False,...,False,False,False,False,True,other,NaN,Commercial,MA,White
2,3,2007-06-07,MEDFORD,Middlesex County,56.0,white,female,vehicular,False,False,...,NaN,False,NaN,NaN,False,NaN,NaN,Passenger,MA,White
3,4,2007-06-07,MEDFORD,Middlesex County,37.0,white,male,vehicular,False,False,...,NaN,False,NaN,NaN,False,NaN,NaN,Commercial,MA,White
4,5,2007-06-07,EVERETT,Middlesex County,22.0,hispanic,female,vehicular,False,True,...,NaN,False,NaN,NaN,False,NaN,NaN,Commercial,MA,Hispanic


We check the data types to see if appropriate types are being used. We will convert them accordinly.

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', data.shape[0]+1)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3416238 entries, 0 to 3416237
Data columns (total 24 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   raw_row_number              int64  
 1   date                        object 
 2   location                    object 
 3   county_name                 object 
 4   subject_age                 float64
 5   subject_race                object 
 6   subject_sex                 object 
 7   type                        object 
 8   arrest_made                 object 
 9   citation_issued             object 
 10  warning_issued              object 
 11  outcome                     object 
 12  contraband_found            object 
 13  contraband_drugs            object 
 14  contraband_weapons          object 
 15  contraband_alcohol          bool   
 16  contraband_other            object 
 17  frisk_performed             object 
 18  search_conducted            bool   
 19  search_basis         

We convert the date from object to date type and extract time for future analyses.

In [5]:
##Convert date to datetype
data.date
data["date"] = pd.to_datetime(data.date, format="%Y-%m-%d")

In [6]:
data['date'] = pd.to_datetime(data['date'])
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day

print(data['month'].head)
data.drop(['date'], axis=1, inplace = True)

<bound method NDFrame.head of 0           6
1           6
2           6
3           6
4           6
           ..
3416233    12
3416234    12
3416235     1
3416236     1
3416237     1
Name: month, Length: 3416238, dtype: int64>


In [7]:
data.head()

,raw_row_number,location,county_name,subject_age,subject_race,subject_sex,type,arrest_made,citation_issued,warning_issued,...,frisk_performed,search_conducted,search_basis,reason_for_stop,vehicle_type,vehicle_registration_state,raw_Race,year,month,day
0,1,MIDDLEBOROUGH,Plymouth County,33.0,white,male,vehicular,False,True,False,...,NaN,False,NaN,Speed,Passenger,MA,White,2007,6,6
1,2,SEEKONK,Bristol County,36.0,white,male,vehicular,False,False,True,...,False,True,other,NaN,Commercial,MA,White,2007,6,7
2,3,MEDFORD,Middlesex County,56.0,white,female,vehicular,False,False,True,...,NaN,False,NaN,NaN,Passenger,MA,White,2007,6,7
3,4,MEDFORD,Middlesex County,37.0,white,male,vehicular,False,False,True,...,NaN,False,NaN,NaN,Commercial,MA,White,2007,6,7
4,5,EVERETT,Middlesex County,22.0,hispanic,female,vehicular,False,True,False,...,NaN,False,NaN,NaN,Commercial,MA,Hispanic,2007,6,7


In [8]:
#Convering race, sex, location etc to categorical
data.iloc[:,np.r_[2:4,5:8,9:12,19:23]]=data.iloc[:,np.r_[2:4,5:8,9:12,19:23]].astype("category")

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3416238 entries, 0 to 3416237
Data columns (total 26 columns):
 #   Column                      Dtype   
---  ------                      -----   
 0   raw_row_number              int64   
 1   location                    object  
 2   county_name                 category
 3   subject_age                 category
 4   subject_race                object  
 5   subject_sex                 category
 6   type                        category
 7   arrest_made                 category
 8   citation_issued             object  
 9   warning_issued              category
 10  outcome                     category
 11  contraband_found            category
 12  contraband_drugs            object  
 13  contraband_weapons          object  
 14  contraband_alcohol          bool    
 15  contraband_other            object  
 16  frisk_performed             object  
 17  search_conducted            bool    
 18  search_basis                object  
 19  

In [10]:
#Checking proportion of missing values
data.isnull().sum()*100/len(data)

raw_row_number                 0.000000
location                       0.195127
county_name                    0.195127
subject_age                    4.625146
subject_race                   0.048709
subject_sex                    0.457316
type                           0.000000
arrest_made                    0.026813
citation_issued                0.026813
warning_issued                 0.026813
outcome                        0.200747
contraband_found              98.368673
contraband_drugs              98.368673
contraband_weapons            98.368673
contraband_alcohol             0.000000
contraband_other              98.368673
frisk_performed               98.400843
search_conducted               0.000000
search_basis                  98.506281
reason_for_stop               48.579432
vehicle_type                   0.145277
vehicle_registration_state     0.287275
raw_Race                       0.048709
year                           0.000000
month                          0.000000


<!-- We will use the KNN Imputer for imputing values into the Age and Sex variables for better precision -->

We remove nulls from the race column since they constitute only 0.048% of the data for better accuracy.

In [11]:
#Removing null, and unknown values from race
data=data[-data["subject_race"].isin(["other","unknown",np.nan])]
data["subject_race"].unique()

array(['white', 'hispanic', 'black', 'asian/pacific islander'],
      dtype=object)

In [12]:
data.nunique()

raw_row_number                3386549
location                          350
county_name                        14
subject_age                        85
subject_race                        4
subject_sex                         2
type                                1
arrest_made                         2
citation_issued                     2
warning_issued                      2
outcome                             3
contraband_found                    2
contraband_drugs                    2
contraband_weapons                  2
contraband_alcohol                  2
contraband_other                    2
frisk_performed                     2
search_conducted                    2
search_basis                        3
reason_for_stop                     7
vehicle_type                        5
vehicle_registration_state         51
raw_Race                            5
year                                9
month                              12
day                                31
dtype: int64

In [13]:
# Removing row number 
data.drop(['raw_row_number'], axis=1, inplace = True)

Encoding categorical variables for future analyses

In [16]:
data.dtypes

date                          datetime64[ns]
location                            category
county_name                         category
subject_age                          float64
subject_race                        category
subject_sex                         category
type                                category
arrest_made                           object
citation_issued                     category
warning_issued                      category
outcome                             category
contraband_found                      object
contraband_drugs                      object
contraband_weapons                    object
contraband_alcohol                      bool
contraband_other                      object
frisk_performed                       object
search_conducted                        bool
search_basis                        category
reason_for_stop                     category
vehicle_type                        category
vehicle_registration_state          category
raw_Race  

In [14]:
data.select_dtypes(include=["object"]).nunique()

location              350
subject_race            4
citation_issued         2
contraband_drugs        2
contraband_weapons      2
contraband_other        2
frisk_performed         2
search_basis            3
dtype: int64

### Encoding categorical variables

We use the get_dummies function to convert categorical variables to one hot encoded indicator variables

In [ ]:
#One hot encoding categorical variables

pd.get_dummies(data.loc[:,['subject_race','subject_sex','arrest_made','citation_issued','outcome','contraband_found','contraband_drugs','contraband_weapons','contraband_alcohol','contraband_other','search_basis','reason_for_stop']])

In [ ]:
pd.get_dummies(data).head()

In [16]:
# Before One-Hot Encoding, let's create separate indep. X variables from dep. Y variable
X = data.drop('subject_race', axis = 1).copy()
# alternatively, X = df.iloc[:, :-1]
y = data['subject_race'].copy()

In [17]:
X_enc = pd.get_dummies(X)
display(X_enc.head())
display(X.shape)
display(X_enc.shape)

,contraband_alcohol,search_conducted,year,month,day,location_ABINGTON,location_ACTON,location_ACUSHNET,location_ADAMS,location_AGAWAM,...,vehicle_registration_state_WV,vehicle_registration_state_WY,raw_Race_A,raw_Race_American Indian or Alaskan Native,raw_Race_Asian or Pacific Islander,raw_Race_Black,raw_Race_Hispanic,raw_Race_Middle Eastern or East Indian (South Asian),raw_Race_None - for no operator present citations only,raw_Race_White
0,False,False,2007,6,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,False,True,2007,6,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,False,False,2007,6,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,False,False,2007,6,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,False,False,2007,6,7,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


(3386549, 24)

(3386549, 552)

In [18]:
# Make sure to convert y dep. variable to from categorical to continous
display(y.value_counts())

white                     2529780
black                      351610
hispanic                   338317
asian/pacific islander     166842
Name: subject_race, dtype: int64

In [19]:
# encode string class values as integers
label_encoder = LabelEncoder()
label_encoder = label_encoder.fit(y)
label_encoded_y = label_encoder.transform(y)

In [20]:
# Checking if the data is imbalanced or not
sum(label_encoded_y)/ len(label_encoded_y)

2.5446506163058618

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, label_encoded_y, random_state = 42, stratify = label_encoded_y)
print("train ratio =", sum(y_train)/ len(y_train))
print("test ratio =", sum(y_test)/ len(y_test))


train ratio = 2.5446501865616553
test ratio = 2.544651905536959


In [ ]:
# Creating the XGBClassfier shell
clf_xgb = xgb.XGBClassifier(objective='multi:softprob', missing=None, seed=42)
# Adding the data and fitting the model
# Instead of finding the optimal # of tress using K-Cross Validation, let's use early stopping to stop the tree when the cost function no longer reduces
# XGBoost will do the cross validation for us; we just have to specifiy the # of rounds to exhaust with no improvement before stopping
# We use evaluation metric as Area Under Precision-Recall Curve
clf_xgb.fit(X_train,
            y_train,
            verbose=True,
            early_stopping_rounds=10,
            eval_metric='aucpr',
            eval_set=[(X_test, y_test)])

<!-- We create a new binomial variable that tells us if the stop was before or after sunset -->